In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_nonweighted_simple_coo_2020.npz").tocsr()

In [2]:
%load_ext Cython

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

np.random.seed(41148)
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [4]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
RP3brecommender= RP3betaRecommender(URM_train)
RP3brecommender.fit(topK=770, alpha=0.414615342821205, beta=0.17823717015919388, normalize_similarity=False)

RP3betaRecommender: URM Detected 82 (1.03 %) cold users.
RP3betaRecommender: URM Detected 2372 (9.13 %) cold items.


In [5]:
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
ItemKNNCBFrecommender=ItemKNNCBFRecommender(URM_train, ICM_all)
ItemKNNCBFrecommender.fit(topK= 227, shrink=664, similarity='cosine', normalize=False, feature_weighting='BM25')

ItemKNNCBFRecommender: URM Detected 82 (1.03 %) cold users.
ItemKNNCBFRecommender: URM Detected 2372 (9.13 %) cold items.
Similarity column 25975 ( 100 % ), 4585.47 column/sec, elapsed time 0.09 min


In [6]:
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
ItemKNNCFrecommender=ItemKNNCFRecommender(URM_train)
ItemKNNCFrecommender.fit(topK=210, shrink=52, similarity='tversky', normalize=True, tversky_alpha=0.5985566775975192, tversky_beta=1.6463076214540535)

ItemKNNCFRecommender: URM Detected 82 (1.03 %) cold users.
ItemKNNCFRecommender: URM Detected 2372 (9.13 %) cold items.
Similarity column 25975 ( 100 % ), 6686.18 column/sec, elapsed time 0.06 min


In [10]:
def WeightGenerator(length,total_sum=100):
    if length == 1:
        yield (total_sum,)
    else:
        for value in range(0,total_sum + 1, 10):
            for permutation in WeightGenerator(length - 1, total_sum - value):
                yield (value,) + permutation

In [12]:
from KNN.WeightedSimilarityHybridRecommender import WeightedSimilarityHybridRecommender

similarities=[ RP3brecommender.W_sparse, ItemKNNCBFrecommender.W_sparse, ItemKNNCFrecommender.W_sparse]
recommender=WeightedSimilarityHybridRecommender(URM_train,similarities)
evaluator_validation = EvaluatorHoldout(URM_valid, cutoff_list=[10])
resultDF= pd.DataFrame({"topK":[],"weights":[],"MAP":[]})

for topK in range(700,1501,50):
    for weights in list(WeightGenerator(3)):
        recommender.fit(np.array(weights)/100,topK)
        MAP,_=evaluator_validation.evaluateRecommender(recommender)
        MAP=MAP[10]["MAP"]
        resultDF=resultDF.append({"topK":[topK],"weights":[np.array(weights)/100],"MAP":[MAP]},ignore_index=True)
        print(topK,weights, MAP)

WeightedSimilarityHybridRecommender: URM Detected 82 (1.03 %) cold users.
WeightedSimilarityHybridRecommender: URM Detected 2372 (9.13 %) cold items.
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.07 sec. Users per second: 1843
700 (0, 0, 100) 0.051562307191321384
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.30 sec. Users per second: 1713
700 (0, 10, 90) 0.06886989893902415
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.30 sec. Users per second: 1713
700 (0, 20, 80) 0.06391567416614335
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.30 sec. Users per second: 1716
700 (0, 30, 70) 0.05663193376919294
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.25 sec. Users per second: 1740
700 (0, 40, 60) 0.050561046169952474
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.11 sec. Users per second: 1820
700 (0, 50, 50) 0.04618213262560332
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.10 sec. Users per second: 1825
700 (0, 60, 40) 0.04288295920571981
EvaluatorHoldout: Processed 5

In [13]:
resultDF_sorted=resultDF.sort_values(by=['MAP'],ascending=False)
resultDF_sorted.to_csv("Hybrid_Weighted_Similarity_results_sorted.csv")

In [16]:
RP3brecommender_full= RP3betaRecommender(URM_all)
RP3brecommender_full.fit(topK=770, alpha=0.414615342821205, beta=0.17823717015919388, normalize_similarity=False)


ItemKNNCBFrecommender_full=ItemKNNCBFRecommender(URM_all, ICM_all)
ItemKNNCBFrecommender_full.fit(topK= 227, shrink=664, similarity='cosine', normalize=False, feature_weighting='BM25')

ItemKNNCFrecommender_full=ItemKNNCFRecommender(URM_all)
ItemKNNCFrecommender_full.fit(topK=210, shrink=52, similarity='tversky', normalize=True, tversky_alpha=0.5985566775975192, tversky_beta=1.6463076214540535)

similarities_full=[RP3brecommender_full.W_sparse, ItemKNNCBFrecommender_full.W_sparse, ItemKNNCFrecommender_full.W_sparse]
recommender_full= WeightedSimilarityHybridRecommender(URM_all, similarities_full)
recommender_full.fit(np.array([0.5,0.4,0.1]), topK=1000)

RP3betaRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCBFRecommender: URM Detected 1079 (4.15 %) cold items.
Similarity column 25975 ( 100 % ), 3967.51 column/sec, elapsed time 0.11 min
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
Similarity column 25975 ( 100 % ), 5165.73 column/sec, elapsed time 0.08 min
WeightedSimilarityHybridRecommender: URM Detected 1079 (4.15 %) cold items.


In [17]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)

for uid in target_user_list:
    rec_list=map(str,recommender_full.recommend(uid)[:10])
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(rec_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("Hybrid_Weighted_Similarity_recommendations.csv")